In [1]:
import invisible_cities.reset.reset_utils as reset

from invisible_cities.core.system_of_units import pes, mm, mus, ns
import invisible_cities.reco.corrections    as corrf
import invisible_cities.database.load_db as dbf 
import reset_functions_event as rstf

# Initialize RESET
iterations = 100
pitch      = 10
run_number = 4495
nsipms     = 1792
npmts      = 1 
dist       = 20. 
sipm_dist  = 20. 
pmt_dist   = 200 # max distance included in the param file
sipm_thr   = 5.
#x_size     = 2.
#y_size     = 2.

x_size     = 10.
y_size     = 10.


rmax       = 198 #
slice_width = 2.
pmt_param  = "/home/jmbenlloch/reset_data/mapas/PMT_Map_corr_keV.h5"
sipm_param = "/home/jmbenlloch/reset_data/mapas/SiPM_Map_corr_z0.0_keV.h5"

pmap_file = '4495_21215.h5'

pmap_conf = {}
pmap_conf['s1_emin'] = 54 * pes 
pmap_conf['s1_emax'] = 2400. * pes 
pmap_conf['s1_wmin'] = 7*25 * ns
pmap_conf['s1_wmax'] = 16*25 * ns
pmap_conf['s1_hmin'] = 12. * pes 
pmap_conf['s1_hmax'] = 400. * pes 
pmap_conf['s1_num'] = 1 
pmap_conf['s1_ethr'] = 0.5 * pes 

pmap_conf['s2_emin'] = 200000. * pes 
pmap_conf['s2_emax'] = 2.e6 * pes 
pmap_conf['s2_wmin'] = 4.* mus 
pmap_conf['s2_wmax'] = 500. * mus 
pmap_conf['s2_hmin'] = 600. * pes 
pmap_conf['s2_hmax'] = 60000. * pes 
pmap_conf['s2_numMin'] = 1 
pmap_conf['s2_numMax'] = 3 
pmap_conf['s2_ethr'] = 0. * pes 
pmap_conf['s2_nsipmmin'] = 1 
pmap_conf['s2_nsipmmax'] = 1792

# Read file and select peaks
selector = reset.refresh_selector(pmap_conf)
s1s, s2s, s2sis, peaks = reset.load_and_select_peaks(pmap_file, 21215, selector)

# Lifetime correction
ZCorr = corrf.LifetimeCorrection(1093.77, 23.99)

# Sensors info
data_sipm = dbf.DataSiPM(run_number)

# Take the first peak
peak = next(iter(peaks))
evt = 21215
sipm_thr = sipm_thr

# Prepare data
voxels_data, slices, energies, zs = reset.prepare_data(s1s, s2s, s2sis, slice_width, evt, peak, data_sipm, nsipms, sipm_thr, dist, ZCorr)
slices_start = reset.slices_start(voxels_data, x_size, y_size)


#rst = rstf.RESET(run_number, nsipms, npmts, dist, sipm_dist, pmt_dist, sipm_thr, x_size, y_size, rmax, sipm_param, pmt_param)
#voxels, slices = rst.run(voxels, slices, energies, slices_start, iterations)
#reset.write_hdf5(voxels, slices, zs) 
#rst._destroy_context()

# Create voxels

In [2]:
import numpy as np

In [3]:
def create_voxels(voxels_data, slices_start, xsize, ysize, rmax):
    max_voxels = slices_start[-1]
    nslices = voxels_data.xmin.shape[0]

    nvoxels = 0 

    voxels_dt  = np.dtype([('x', 'f4'), ('y', 'f4'), ('E', 'f4')])
    voxels     = np.empty(max_voxels, dtype=voxels_dt)
    slice_ids  = np.empty(max_voxels, dtype='i4')
    address    = np.empty(max_voxels, dtype='i4')

    slice_start_voxels = np.zeros(nslices+1, dtype='i4')

    for slice_id in range(nslices):
        xmin   = voxels_data.xmin[slice_id];
        xmax   = voxels_data.xmax[slice_id];
        ymin   = voxels_data.ymin[slice_id];
        ymax   = voxels_data.ymax[slice_id];
        charge = voxels_data.charge[slice_id];
        xsteps = (xmax - xmin) / xsize + 1;
        ysteps = (ymax - ymin) / ysize + 1;

        xs = np.linspace(xmin, xmax, xsteps)
        ys = np.linspace(ymin, ymax, ysteps)

        voxels_nc = np.array([(x, y, charge) for x in xs for y in ys], dtype=voxels_dt)
        check_actives = np.vectorize(lambda v: np.sqrt(v[0]**2 + v[1]**2) < rmax, otypes=[bool])
        actives = check_actives(voxels_nc)
        nactive = actives.sum()
        start = slice_start_voxels[slice_id]
        end = start + nactive
        voxels[start:end] = voxels_nc[actives]
        slice_ids[start:end] = slice_id

        slice_start_voxels[slice_id+1] = end 
        nvoxels = nvoxels + nactive

        print(slice_id, nactive, end)

        address[slices_start[slice_id]:slices_start[slice_id+1]] = actives

    address = address.cumsum()

    reset_voxels = ResetVoxels(nvoxels, voxels[:nvoxels],
                               slice_ids[:nvoxels], slice_start_voxels,
                               address)

    return reset_voxels

In [4]:
nvoxels, voxels, slice_ids, slice_start, address = create_voxels(voxels_data, slices_start, x_size, y_size, rmax)

/home/jmbenlloch/miniconda3/envs/IC3.6/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
/home/jmbenlloch/miniconda3/envs/IC3.6/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


0 67 67
1 106 173
2 297 470
3 402 872
4 532 1404
5 313 1717
6 485 2202
7 911 3113
8 1026 4139
9 996 5135
10 1104 6239
11 893 7132
12 771 7903
13 509 8412
14 297 8709
15 248 8957
16 234 9191
17 152 9343
18 136 9479
19 121 9600
20 41 9641
21 39 9680
22 39 9719
23 49 9768
24 66 9834
25 79 9913
26 103 10016
27 53 10069
28 46 10115
29 41 10156
30 67 10223
31 80 10303
32 793 11096
33 75 11171
34 84 11255
35 75 11330
36 75 11405
37 63 11468
38 69 11537
39 120 11657
40 63 11720
41 85 11805
42 63 11868
43 69 11937
44 75 12012
45 76 12088
46 87 12175
47 84 12259
48 74 12333
49 181 12514
50 87 12601
51 77 12678
52 77 12755
53 72 12827
54 71 12898
55 55 12953
56 59 13012
57 47 13059
58 36 13095
59 28 13123
60 47 13170
61 34 13204
62 40 13244
63 54 13298
64 286 13584
65 242 13826


NameError: name 'ResetVoxels' is not defined

In [ ]:
xsize = x_size
ysize = y_size

max_voxels = slices_start[-1]
nslices = voxels_data.xmin.shape[0]

nvoxels = 0 

voxels_dt  = np.dtype([('x', 'f4'), ('y', 'f4'), ('E', 'f4')])
voxels     = np.empty(max_voxels, dtype=voxels_dt)
slice_ids  = np.empty(max_voxels, dtype='i4')
#address    = np.empty(max_voxels, dtype='i4')
address    = np.zeros(max_voxels, dtype='i4')

slice_start_voxels = np.zeros(nslices+1, dtype='i4')

count = 0

#for slice_id in range(nslices):
for slice_id in [8]:
    xmin   = voxels_data.xmin[slice_id];
    xmax   = voxels_data.xmax[slice_id];
    ymin   = voxels_data.ymin[slice_id];
    ymax   = voxels_data.ymax[slice_id];
    charge = voxels_data.charge[slice_id];
    xsteps = (xmax - xmin) / xsize + 1;
    ysteps = (ymax - ymin) / ysize + 1;

    xs = np.linspace(xmin, xmax, xsteps)
    ys = np.linspace(ymin, ymax, ysteps)

    voxels_nc = np.array([(x, y, charge) for x in xs for y in ys], dtype=voxels_dt)
    check_actives = np.vectorize(lambda v: np.sqrt(v[0]**2 + v[1]**2) < rmax, otypes=[bool])
    actives = check_actives(voxels_nc)
    nactive = actives.sum()
    start = slice_start_voxels[slice_id]
    end = start + nactive
    voxels[start:end] = voxels_nc[actives]
    slice_ids[start:end] = slice_id

    slice_start_voxels[slice_id+1] = end 
    nvoxels = nvoxels + nactive

    print("{}\t{}\t{}\t{}\t{}\t".format(slice_id, count, nactive, start, end))
    count = count + nactive

    address[slices_start[slice_id]:slices_start[slice_id+1]] = actives
#    print(address[slices_start[slice_id]:slices_start[slice_id+1]].sum())

address = address.cumsum()

In [ ]:
actives

In [ ]:
address[slice_start[slice_id]:slice_start[slice_id+1]] + 2203

In [195]:
actives.sum()

911

# Anode response

In [7]:
def create_anode_response(nslices, nsensors, slices):
    total_sensors = nslices * nsensors
    anode_response = np.zeros(total_sensors, dtype='f4')

    slice_id = 0
    ncharges = slices.charges.shape[0]

    for s in range(ncharges):
        if s >= slices.start[slice_id+1]:
            slice_id = slice_id + 1

        position = slices.sensors[s] + nsensors * slice_id
        anode_response[position] = slices.charges[s]
    
    return anode_response

In [8]:
anode_response[1023]

2.6653137

In [6]:
nslices = voxels_data.xmin.shape[0]
anode_response = create_anode_response(nslices, nsipms, slices)

# Create cathode response

In [43]:
energies

array([7.2264624e+02, 4.1959043e+03, 1.0519784e+04, 1.7254588e+04,
       1.8523502e+04, 1.6994426e+04, 2.0278189e+04, 2.7795934e+04,
       4.0335648e+04, 5.2568531e+04, 5.2349938e+04, 3.8738012e+04,
       2.6826695e+04, 2.3684562e+04, 2.2167107e+04, 1.8603082e+04,
       1.3202256e+04, 8.4173896e+03, 5.6608184e+03, 3.5756594e+03,
       2.5694243e+03, 2.6684583e+03, 2.1014158e+03, 2.2701426e+03,
       2.9455864e+03, 4.9063867e+03, 5.2197192e+03, 4.2988892e+03,
       2.9707058e+03, 2.2736929e+03, 2.2044587e+03, 2.7128088e+03,
       3.9867952e+03, 5.7572329e+03, 5.8722012e+03, 4.9187495e+03,
       3.6355945e+03, 3.1717122e+03, 3.4265986e+03, 4.5101162e+03,
       5.6360449e+03, 5.3188843e+03, 3.5691736e+03, 3.8035217e+03,
       3.9004221e+03, 4.5042705e+03, 4.7847246e+03, 5.0263403e+03,
       5.1779131e+03, 4.5697681e+03, 3.8743059e+03, 4.2185522e+03,
       4.8349048e+03, 4.8407236e+03, 4.1809199e+03, 3.4839041e+03,
       3.0245417e+03, 2.2185493e+03, 1.7951973e+03, 1.6025044e

# Compute active sensors

In [44]:
import math

In [45]:
import invisible_cities.reco.dst_functions as dstf

pmt_param_file   = "/home/jmbenlloch/reset_data/mapas/PMT_Map_corr_keV.h5"
sipm_param_file  = "/home/jmbenlloch/reset_data/mapas/SiPM_Map_corr_z0.0_keV.h5"

sipm_param = dstf.load_xy_corrections(sipm_param_file, group="ResetMap", node="SiPM")
#dstf.load_xy_corrections(pmtcorr+".h5", group="ResetMap", node="PMT")

In [46]:
sipms_per_voxel = int(math.floor(2 * sipm_dist / pitch) + 1)**2
voxels_per_sipm = int((2 * sipm_dist)**2 / (x_size * y_size))
xs = data_sipm.X
ys = data_sipm.Y

In [145]:
def compute_probabilities(voxels, nvoxels, xs, ys, nsensors, sensors_per_voxel, sensor_dist, sensor_param, sensor_response):
    probs_size = nvoxels * sensors_per_voxel
    sensor_ids = np.empty(probs_size, dtype='i4')
    probs      = np.empty(probs_size, dtype='f4')
    fwd_num    = np.empty(probs_size, dtype='f4')
    voxel_starts = np.zeros(nvoxels+1, dtype='i4')

    sensor_starts = np.zeros(nsensors * nslices + 1, dtype='i4')

    last_position = 0
    for i, v in enumerate(voxels):
        for s in range(nsensors):
            xdist = v[0] - xs[s]
            ydist = v[1] - ys[s]
            active = ((abs(xdist) <= sensor_dist) and (abs(ydist) <= sensor_dist))
            if active:
                #print(v, s, xdist, ydist)
                sens_idx = slice_ids[i] * nsensors + s
                sensor_starts[sens_idx + 1] = sensor_starts[sens_idx + 1] + 1
                
                #sensor_ids[last_position] = s
                sensor_ids[last_position] = sens_idx
                prob                      = sensor_param(xdist, ydist).value
                probs     [last_position] = prob
                fwd_num   [last_position] = prob * sensor_response[sens_idx]
                last_position = last_position + 1

                
        voxel_starts[i+1] = last_position
    
    sensor_starts = sensor_starts.cumsum()
        
    return probs[:last_position], sensor_ids[:last_position], voxel_starts, sensor_starts, last_position, fwd_num[:last_position]

In [146]:
probs, sensor_ids, voxel_starts, sensor_starts, nprobs, fwd_num = compute_probabilities(voxels[:1000], 1000, xs, ys, nsipms, sipms_per_voxel, sipm_dist, sipm_param, anode_response)

In [147]:
sensor_ids

array([1352, 1360, 1368, ..., 8285, 8286, 8287], dtype=int32)

# Sensor probs

In [49]:
nsensors = nsipms

In [148]:
# offset one position to get the ending position of the last element
sensor_counts = np.zeros(nsensors * nslices + 1, dtype='i4')
voxel_ids    = np.empty(nprobs, dtype='i4')
sensor_probs = np.empty(nprobs, dtype='f4')

vidx = 0
for i, p in enumerate(probs):
    if i >= voxel_starts[vidx + 1]:
        vidx = vidx + 1
    
    sidx     = sensor_ids[i]
    slice_id = slice_ids[vidx]
    
    count = sensor_counts[sidx + 1]
    pos   = sensor_starts[sidx] + count
    
    sensor_probs[pos] = p
    voxel_ids[pos] = vidx
    
    sensor_counts[sidx + 1] = count + 1
    
active_sensors = sensor_counts > 0
sensor_starts_ids = np.where(active_sensors)[0] - 1
sensor_starts_c = sensor_counts.cumsum()[sensor_starts_ids]

In [149]:
def compute_sensor_probs(probs, nprobs, nsensors, voxel_starts, sensor_ids, slice_ids):
    # offset one position to get the ending position of the last element
    sensor_counts = np.zeros(nsensors * nslices + 1, dtype='i4')
    voxel_ids    = np.empty(nprobs, dtype='i4')
    sensor_probs = np.empty(nprobs, dtype='f4')

    vidx = 0
    for i, p in enumerate(probs):
        if i >= voxel_starts[vidx + 1]:
            vidx = vidx + 1

        sidx     = sensor_ids[i]
        slice_id = slice_ids[vidx]

        count = sensor_counts[sidx + 1]
        pos   = sensor_starts[sidx] + count

        sensor_probs[pos] = p
        voxel_ids[pos] = vidx

        sensor_counts[sidx + 1] = count + 1

    active_sensors = sensor_counts > 0
    sensor_starts_c_ids = np.where(active_sensors)[0] - 1
    sensor_starts_c = sensor_counts.cumsum()[np.concatenate((sensor_starts_c_ids, [-1]))]
    
    return sensor_probs, sensor_starts, sensor_starts_c, sensor_starts_c_ids

In [150]:
sensor_probs, sensor_starts, sensor_starts_c, sensor_starts_c_ids = compute_sensor_probs(probs, nprobs, nsipms, voxel_starts, sensor_ids, slice_ids)

In [151]:
sensor_starts_c_ids.shape

(1662,)

In [152]:
sensor_starts

array([    0,     0,     0, ..., 24958, 24958, 24958])

In [153]:
sensor_starts_ids.shape

(1662,)

In [154]:
sensor_starts_c.shape

(1663,)

In [155]:
sensor_starts_c

array([    0,     1,     3, ..., 24953, 24955, 24958])

# Forward denom

In [156]:
def forward_denoms(nsensors, nslices, voxels, sensor_probs, voxel_ids, sensor_starts, sensor_starts_ids):
    nsensor_active = sensor_starts_ids.shape[0]
    denoms = np.zeros(nsensors * nslices, dtype='f4')

    for id in np.arange(nsensor_active):
        start = sensor_starts[id]
        end   = sensor_starts[id+1]

        denom = 0.
        for i in np.arange(start, end):
            vid = voxel_ids[i]
            denom += voxels[vid][2] * sensor_probs[i]

        sid = sensor_starts_c_ids[id]
        denoms[sid] = denom

    return denoms

In [157]:
forward_denoms(nsipms, nslices, voxels, sensor_probs, voxel_ids, sensor_starts_c, sensor_starts_c_ids)

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

# MLEM step

In [163]:
def mlem_step(voxels, nvoxels, voxel_starts, probs, sensor_ids, fwd_num, fwd_denoms):
    #for vidx in np.arange(nvoxels):
    for vidx in np.arange(1000):
        start = voxel_starts[vidx]
        end   = voxel_starts[vidx+1]

        eff = 0
        fwd = 0
        for i in np.arange(start, end):
            eff += probs[i]
            sidx = sensor_ids[i]

            value = fwd_num[i] / denoms[sidx]

            #TODO check is finite
            if(np.isfinite(value)):
                fwd += value

        voxels[vidx][2] = voxels[vidx][2] / eff * fwd

# check probs

In [ ]:
import pandas as pd
sipm_corr = pd.read_hdf("/home/jmbenlloch/reset_data/mapas/SiPM_Map_corr_z0.0_keV.h5")

In [ ]:
factors = sipm_corr.factor.values

In [ ]:
value = 0.18359876
np.abs(factors - value).argmin()

In [ ]:
sipm_corr[3009:3011]

In [ ]:
value_cuda = 0.18654083
np.abs(factors - value_cuda).argmin()

In [ ]:
import scipy
scipy.interpolate.NearestNDInterpolator(1)

In [ ]:
xs_param = np.linspace(-29.5, 29.5, 60)
ys_param = np.linspace(-29.5, 29.5, 60)

In [ ]:
coords = np.array([(x, y) for x in xs_param for y in ys_param])

In [ ]:
factors.shape

In [ ]:
coords.shape

In [ ]:
interp = scipy.interpolate.NearestNDInterpolator(coords, np.arange(coords.shape[0]))

In [ ]:
interp(20, -20)